In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
# importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
heart_data = pd.read_csv('/content/heart (1).csv')
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
heart_data.shape

(303, 14)

In [7]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [8]:
# checking the distribution of Target Variable
heart_data['target'].value_counts()

target
1    165
0    138
Name: count, dtype: int64

In [9]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [10]:
X = np.asarray(X)
Y = np.asarray(Y)

Model Selection

Comparing the models with default hyperparameter values using Cross Validation

In [18]:
models=[LogisticRegression(max_iter=1000),SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier(random_state=2)]

In [23]:
def compare_models_crossValidation():

  for model in models:

    cv_score = cross_val_score(model, X, Y, cv=5)
    mean_accuracy = sum(cv_score)/len(cv_score)
    mean_accuracy = mean_accuracy*100
    mean_accuracy = round(mean_accuracy, 2)

    print('Cross Validation accuracies for the',model,'=', cv_score)
    print('Acccuracy score of the ',model,'=',mean_accuracy,'%')
    print('---------------------------------------------------------------')

In [22]:
compare_models_crossValidation()

Cross Validation accuracies for the model  LogisticRegression(max_iter=1000) :  [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy for the model  LogisticRegression(max_iter=1000)  is :  82.83 %
-----------------------------------------------------------
Cross Validation accuracies for the model  SVC(kernel='linear') :  [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy for the model  SVC(kernel='linear')  is :  82.83 %
-----------------------------------------------------------
Cross Validation accuracies for the model  KNeighborsClassifier() :  [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy for the model  KNeighborsClassifier()  is :  64.39 %
-----------------------------------------------------------
Cross Validation accuracies for the model  RandomForestClassifier(random_state=2) :  [0.85245902 0.8852459  0.81967213 0.81666667 0.76666667]
Accuracy for the model  RandomForestClassifier(random_state=2)  is :  82.81 %
------------------

# Comparing the models with different Hyperparameter values using GridSearchCV

In [25]:
models_list = [LogisticRegression(max_iter=10000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [43]:
#creating a dictionary that contains hyperparameter values for above models
hyperparameters={
    'log_reg_params':{
        'C':[1,5,10,20]
    },
    'svc_hyperparams':{
        'kernel':['linear','rbf','poly','sigmoid'],
        'C':[1,5,10,20]
    },
    'knn_hyperparams':{
        'n_neighbors':[3,5,10]
    },
    'random_forest_hyperparams':{
        'n_estimators':[10,20,50,100]
    }
}

In [31]:
type(hyperparameters)

dict

In [32]:
print(hyperparameters.keys())

dict_keys(['log_reg_params', 'svc_hyperparams', 'knn_hyperparams', 'random_forest_hyperparams'])


In [37]:
print(hyperparameters.values())

dict_values([{'C': [1, 5, 10, 20]}, {'kernel': {'rbf', 'poly', 'sigmoid', 'linear'}, 'C': [1, 5, 10, 20]}, {'n_neighbors': [3, 5, 10]}, {'n_estimators': [10, 20, 50, 100]}])


In [33]:
hyperparameters['log_reg_params']

{'C': [1, 5, 10, 20]}

In [34]:
model_keys=list(hyperparameters.keys())
print(model_keys)

['log_reg_params', 'svc_hyperparams', 'knn_hyperparams', 'random_forest_hyperparams']


In [35]:
model_keys[0]

'log_reg_params'

In [36]:
hyperparameters[model_keys[1]]

{'kernel': {'linear', 'poly', 'rbf', 'sigmoid'}, 'C': [1, 5, 10, 20]}

In [41]:
def ModelSelection(models_list,hyperparameters):

  #we will store all the best hyperparameters in this
  result=[]

  i=0

  for model in models_list:
    key=model_keys[i]
    params=hyperparameters[key]
    i+=1
    print(model)
    print(params)
    print('----------------------------')

    classifier=GridSearchCV(model,params,cv=5)
    classifier.fit(X,Y)

    result.append({
        'model used':model,
        'highest score':classifier.best_score_,
        'best hyperparamters':classifier.best_params_
    })

  result_df = pd.DataFrame(result,columns=['model used','highest score','best hyperparamters'])
  return result_df

In [44]:
ModelSelection(models_list,hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
----------------------------
SVC()
{'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 'C': [1, 5, 10, 20]}
----------------------------
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
----------------------------
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
----------------------------


,model used,highest score,best hyperparamters
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}
